In [1]:
# Importamos librerías necesarias

import requests
import pandas as pd
import psycopg2
from datetime import datetime

In [2]:
# URL base de la API de CoinCap
base_url = "https://api.coincap.io/v2"


In [3]:
# Endpoint para obtener datos de activos
endpoint = "/assets"

In [4]:
# Encabezados de la solicitud con la clave API
headers = {
    "Accept-Encoding": "gzip, deflate"}

In [5]:
# URL completa para la solicitud
url = base_url + endpoint

In [6]:
# Realizar la solicitud GET a la API
response = requests.get(url, headers=headers)

In [7]:
# Verificar el código de estado de la respuesta
if response.status_code == 200:
    # La solicitud fue exitosa
    data = response.json()
    # Imprimir los datos de los activos (solo imprime los primeros 5 en este ejemplo)
    for asset in data['data'][:5]:
        print(asset)
else:
    # La solicitud falló
    print("Error al realizar la solicitud:", response.status_code)


{'id': 'bitcoin', 'rank': '1', 'symbol': 'BTC', 'name': 'Bitcoin', 'supply': '19658368.0000000000000000', 'maxSupply': '21000000.0000000000000000', 'marketCapUsd': '1379541138576.5233779148320896', 'volumeUsd24Hr': '5248317047.9717464994862041', 'priceUsd': '70175.7713853216796997', 'changePercent24Hr': '0.1435807209903933', 'vwap24Hr': '69945.2007363079532577', 'explorer': 'https://blockchain.info/'}
{'id': 'ethereum', 'rank': '2', 'symbol': 'ETH', 'name': 'Ethereum', 'supply': '120077426.1898597300000000', 'maxSupply': None, 'marketCapUsd': '426013510808.7970615577309671', 'volumeUsd24Hr': '4079956285.5823621395890759', 'priceUsd': '3547.8234696270742473', 'changePercent24Hr': '0.1207086690090205', 'vwap24Hr': '3514.8806637212792651', 'explorer': 'https://etherscan.io/'}
{'id': 'tether', 'rank': '3', 'symbol': 'USDT', 'name': 'Tether', 'supply': '103618183617.7160800000000000', 'maxSupply': None, 'marketCapUsd': '103738958254.0555888473491174', 'volumeUsd24Hr': '15974574796.261312336

In [8]:
# Convertir el JSON a un DataFrame de pandas
df = pd.DataFrame(data['data'])

In [10]:
# Eliminar columnas innecesarias
df.drop(['id','maxSupply', 'vwap24Hr', 'explorer'], axis=1, inplace=True)

In [12]:
# Elegimos la columna symbol como ID. Reordenamos y renombramos
symbol_column = df.pop('symbol')  
df.insert(0, 'id', symbol_column)

In [14]:
#Agregamos columna fecha de carga
fecha_hoy = datetime.now().timestamp()
df['fecha_hoy'] = pd.to_datetime(fecha_hoy, unit='s')

In [16]:
# Creamos la conexión a Redsshift
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user="matiaspereyra_coderhouse"
with open("C:/Users/machi/Desktop/Cursos/Data Engineer/Coder/pwdredshift.txt",'r') as f:
    pwd= f.read()
try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Conectado a Redshift con éxito!")
    
except Exception as e:
    print("No es posible conectar a Redshift")
    print(e)

Conectado a Redshift con éxito!


In [17]:
#Crear cursor
cursor = conn.cursor()

In [18]:
#Crear la tabla
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS matiaspereyra_coderhouse.crypto
        (
	    id VARCHAR(50) primary key  
	    ,rank INTEGER
	    ,name VARCHAR(255)  
	    ,supply DECIMAL
	    ,marketCapUsd DECIMAL
	    ,volumeUsd24Hr DECIMAL  
	    ,priceUsd DECIMAL 
	    ,changePercent24Hr DECIMAL   
	    ,fecha_hoy TIMESTAMP   
	    )
    """)
    conn.commit()

In [19]:
# Eliminar los registros existentes
cursor.execute("DELETE FROM crypto")
conn.commit()

In [20]:
# Insertamos registros
for index, row in df.iterrows():
    cursor.execute("INSERT INTO crypto (id, rank, name, supply, marketCapUsd, volumeUsd24Hr, priceUsd, changePercent24Hr, fecha_hoy) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
                   (row['id'], row['rank'], row['name'], row['supply'], row['marketCapUsd'], row['volumeUsd24Hr'], row['priceUsd'], row['changePercent24Hr'], row['fecha_hoy']))

# Confirmar la inserción de los registros
conn.commit()



In [21]:
# Consulta
query = "SELECT * FROM crypto"

In [22]:
# Leer la tabla en un DataFrame
df_result = pd.read_sql_query(query, conn)

C:\Users\machi\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [23]:
# Cerrar el cursor y la conexión
cursor.close()
conn.close()

In [24]:
# Mostrar el DataFrame resultante
print(df_result)

      id  rank         name        supply  marketcapusd  volumeusd24hr  \
0    BTC     1      Bitcoin  1.965837e+07  1.379541e+12   5.248317e+09   
1    ETH     2     Ethereum  1.200774e+08  4.260135e+11   4.079956e+09   
2   USDT     3       Tether  1.036182e+11  1.037390e+11   1.597457e+10   
3    BNB     4          BNB  1.668011e+08  1.012382e+11   5.552180e+08   
4    SOL     5       Solana  4.438849e+08  8.735368e+10   1.258173e+09   
..   ...   ...          ...           ...           ...            ...   
95   RPL    96  Rocket Pool  2.021717e+07  6.050484e+08   2.942030e+05   
96   FXS    97   Frax Share  7.856994e+07  5.959814e+08   4.471870e+05   
97   HOT    98         Holo  1.731164e+11  5.932524e+08   3.793796e+06   
98   LPT    99     Livepeer  3.136148e+07  5.851024e+08   1.010866e+07   
99    SC   100      Siacoin  5.672512e+10  5.785573e+08   5.020830e+06   

    priceusd  changepercent24hr                  fecha_hoy  
0    70176.0                0.0 2024-03-30 15:18:2